IMPORT LIBRARY

In [4]:
import sys, os
current_dir = os.getcwd()
project_root = current_dir[:current_dir.find("src") - 1]
sys.path.insert(0, project_root)

import pandas as pd
from data_selector import Data_selector
from feature_modifier import Feature_selector, Feature_adder
from logs.logger import CustomLogger
from models import Random_Forest, Linear
from src.root import get_root

logger = CustomLogger(name="model_main", log_file_name='model_main.log').get_logger()

e:\univercity\project\Bargh_ML\logs\model_main.log


READ CSV

In [5]:
def filter_time(df,date1,date2,get_bool=False):
    if get_bool:
        return (df['datetime'] >= date1) & (df['datetime'] <= date2)
    return df[(df['datetime'] >= date1) & (df['datetime'] <= date2)]

def filter_name_code(df,name,code,get_bool=False):
    if get_bool:
        return (df["name"] == name) & (df["code"] == code)
    return df[(df["name"] == name) & (df["code"] == code)]

def get_interval(df,l_min):
    df_s = df.reset_index(drop=True)
    diff = df_s['datetime'].diff()  # محاسبه اختلاف زمانی بین ردیف ها
    gap_mask = diff != pd.Timedelta(hours=1)  # هر جایی اختلاف دقیقاً 1 ساعت نیست، مرز بازه جدید است
    # ایندکس شروع بازه‌ها
    start_indices = df_s.index[gap_mask].tolist()
    # چون اولین ایندکس هم ابتدای یک بازه است، اگر نیست اضافه می‌کنیم
    if 0 not in start_indices:
        start_indices = [0] + start_indices
    # ایندکس پایان بازه‌ها یکی قبل از شروع بازه بعدی است
    end_indices = [i-1 for i in start_indices[1:]] + [df_s.index[-1]]
    
    for i in range(len(start_indices)-1,-1,-1):
        if end_indices[i] - start_indices[i] < l_min-1:
            end_indices.pop(i)
            start_indices.pop(i)
    # ساخت لیست بازه‌های (i1, i2)
    index_ranges = list(zip(start_indices, end_indices))
    
    # ساخت لیست بازه‌های زمانی (t1, t2)
    time_ranges = [(df_s.loc[i1, 'datetime'], df_s.loc[i2, 'datetime']) for i1, i2 in index_ranges]
    return index_ranges,time_ranges

import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

def is_smooth_array(data,threshold,f_show = False):
    x = data["datetime"].to_numpy()  # Your x-values
    y = data["generation"].to_numpy()  # Your y-values
    window_length = len(y) - 1 + len(y)%2  # window size, must be odd, adjust for your data
    
    polyorder = 2  # polynomial order
    y_smooth = savgol_filter(y, window_length=window_length, polyorder=polyorder)

    # Compute residual noise
    residuals = y - y_smooth
    noise_std = np.std(residuals)
    value = np.var(np.diff(y, n=1))#abs(y-y.mean())#residuals
    
    # Optional: Visualize
    if f_show:
        plt.plot(x, y, label='Raw data')
        plt.plot(x, y_smooth, label='Smoothed curve')
        plt.legend()
        plt.show()

    if noise_std < threshold:  # threshold is your chosen value for smoothness
        return noise_std,True,value
    else:
        return noise_std,False,value

def is_smooth(df,date1,date2,data_var,data_date,threshold,flag=False):
    slice = filter_time(df,date1,date2)[["hour","generation","datetime"]]
    if len(slice["hour"]) >= 8:
        x,f,re = is_smooth_array(slice,threshold,flag)
        if data_var != None : data_var[f].append(x)
        if data_date != None : data_date[f].append((date1,date2))
        if f:    
            return 1,1
        return 1,0
    return 0,0

def get_smooth_good_slice(df,time_ranges,threshold):
    data_var = {True:[],False:[]}
    data_date = {True:[],False:[]}
    t = 0
    n = 0
    for date1,date2 in time_ranges:
        nn,tt = is_smooth(df,date1,date2,data_var,data_date,threshold)
        n += nn
        t += tt
    p = t/n*100 if n != 0 else None
    return p,data_var,data_date

def labeling_point(df,df_n_c,date,label):
    for date1,date2 in date:
        flag_array = filter_time(df_n_c,date1,date2,get_bool=True)
        df.loc[flag_array.index[flag_array],"is_good_pick"] = label

In [8]:
df_row = pd.read_csv(get_root() +"/data/processed/integrated.csv", encoding='utf-8')
df_modified = df_row[["name","code","date","hour","status",'value',"generation"]].copy(deep=True)

feature_adder = Feature_adder(df_modified)
feature_adder.add_season()
df_modified = Data_selector(df_modified).select(m_in_summer=True)

threshold = 2

df_modified['datetime'] = df_modified['date'] + pd.to_timedelta(df_row['hour'], unit='h')
df_row["is_good_pick"] = 0

power_plants = df_modified[['name', 'code']].drop_duplicates()
n = len(power_plants)

for _, row in power_plants.iterrows():
    
    df_name_code_smooth = filter_name_code(df_modified,row["name"],row["code"])
    
    index_ranges,time_ranges = get_interval(df_name_code_smooth,l_min=4)
    #print(k,"->",np.unique(np.array([(t2-t1).total_seconds()/3600+1 for t1,t2 in time_ranges]),return_counts=True))
    p,data_var,data_date = get_smooth_good_slice(df=df_name_code_smooth,time_ranges=time_ranges,threshold=threshold)
    if p != 0 and p != None:
        #print("**********")
        print(row["name"],row["code"],p)
        #print(len(data_date[True]),len(data_date[False]))
        #print(k,"->",np.unique(np.array([(t2-t1).total_seconds()/3600+1 for t1,t2 in time_ranges]),return_counts=True))
    
    labeling_point(df_row,df_name_code_smooth,data_date[False],label=1)
    labeling_point(df_row,df_name_code_smooth,data_date[True],label=2)
    
df_row.to_csv(path_or_buf=get_root() +'/data/processed/prediction_only.csv', sep=',', header=True, index=False,na_rep='NULL')
        

پرند G16 37.714285714285715
شهدای پیروز - بهبهان G12 65.3179190751445
شهدای پیروز - بهبهان S1 46.85714285714286
جنوب اصفهان - چهلستون G15 41.95402298850575
سیکل ترکیبی شیروان S1 12.578616352201259
سیکل ترکیبی شیروان G11 8.405797101449275
عسلویه G14 42.35294117647059
عسلویه G15 21.965317919075144
سیکل ترکیبی ارومیه G13 51.445086705202314
سیکل ترکیبی ارومیه G14 52.87356321839081
عسلویه G16 50.29239766081871
پرند G15 38.372093023255815
پرند G14 35.26011560693642
شهدای پیروز - بهبهان G11 76.3586956521739
جنوب اصفهان - چهلستون G11 54.76839237057221
سبلان G11 54.78260869565217
سیکل ترکیبی ارومیه G11 50.71633237822349
سیکل ترکیبی ارومیه G16 25.581395348837212
سیکل ترکیبی شیروان S2 14.545454545454545
سیکل ترکیبی شیروان G12 7.926829268292683
سیکل ترکیبی شیروان G13 8.125
عسلویه G11 62.15469613259669
حافظ G11 46.72131147540984
جنوب اصفهان - چهلستون G14 45.40229885057471
جنوب اصفهان - چهلستون G16 44.57142857142857
سیکل ترکیبی ارومیه G15 24.418604651162788
سیکل ترکیبی ارومیه G12 59.30232558139535
س

In [9]:
np.count_nonzero(df_row["is_good_pick"].to_numpy())

339385

# + MODEL

In [ ]:
import sys, os
current_dir = os.getcwd()
project_root = current_dir[:current_dir.find("src") - 1]
sys.path.insert(0, project_root)

import pandas as pd
from data_selector import Data_selector
from feature_modifier import Feature_selector, Feature_adder
from logs.logger import CustomLogger
from src.models.models import Random_Forest, Linear, Polynomial, XGBoost

logger = CustomLogger(name="model_main", log_file_name='model_main.log').get_logger()

csv_path = os.path.join(project_root, "data", "processed", "prediction_only.csv")
df = pd.read_csv(csv_path, encoding='utf-8')
logger.info(f"Csv file has bean read successfully")

feature_adder = Feature_adder(df)
feature_adder.add_season()
# feature_adder.create_feature_with_delay("temperature", 1)
# feature_adder.create_feature_with_delay("temperature", 2)
# feature_adder.create_feature_with_delay("temperature", 3)
logger.info(f"Some features have been added successfully")

logger.info(f"Rows have been selected successfully")
df_modified = df.copy()
df_modified = Data_selector(df_modified).select(m_in_summer=True)
feature_selector = Feature_selector(df_modified, "generation")
feature_to_be_dropped = ['id', 'date', 'declare', 'require', 'dew', 'apparent_temperature', 'rain', 'snow',
                            'evapotransporation', 'wind_direction']


X, y = feature_selector.select(feature_to_be_dropped)
y = y[X['is_good_pick'] == 2]
X = X[X["is_good_pick"] == 2]

y = y[((X['status'] == 'SO') | (X['status'] == 'LF1'))]
X = X[((X['status'] == 'SO') | (X['status'] == 'LF1'))]

peak_condition = (X['value'] == 'P') | (X['value'] == 'M') & (X['season'] == 'summer')
y = y[peak_condition]
peak_condition = (X['value'] == 'P') | (X['value'] == 'M') & (X['season'] == 'summer')
X = X[peak_condition]

logger.info(f"Some features have been dropped successfully")

In [ ]:
len(df),len(df_modified),len(X)

(1591896, 459689, 80617)

In [51]:
n_est = 40 # 40
depth = 10 # 22
model = Random_Forest()
model.scale_and_split_data(X, y)

train_err = []
test_err = []
#for i in range(1,depth+1): 
model.fit(n_estimators=n_est, max_depth=depth)
mse_train_actual, mse_test_actual = model.compute_mse_error()
print("depth",depth,f"Train Error: {mse_train_actual:0.2f}%",f"Test Error: {mse_test_actual:0.2f}%")
train_err.append(mse_train_actual)
test_err.append(mse_test_actual)

depth 10 Train Error: 3.70% Test Error: 4.04%


In [52]:
csv_path = os.path.join(project_root, "data", "processed", "prediction_only.csv")
df = pd.read_csv(csv_path, encoding='utf-8')
logger.info(f"Csv file has bean read successfully")

feature_adder = Feature_adder(df)
feature_adder.add_season()

df_modified = df.copy(deep=True)
feature_selector = Feature_selector(df_modified, "generation")
feature_to_be_dropped = ['id', 'date', 'declare', 'require', 'dew', 'apparent_temperature', 'rain', 'snow',
                            'evapotransporation', 'wind_direction']

X, y = feature_selector.select(feature_to_be_dropped)

2025-08-19 05:37:49 - model_main - INFO - Csv file has bean read successfully
